# <span style="color:#1f77b4">**Data Analytics 02 - DataFrame Schema**</span>

This notebook shapes raw CSV data into clean, typed DataFrames and summary views.



### <span style="color:#1f77b4">**Loading CSV files into the databricks file system (dbfs)**</span>

Create a working folder in DBFS and download the raw CSV files.

- `%sh` runs shell commands in the notebook context.
- `wget` downloads the CSV files into DBFS.



In [ ]:
# Create a DBFS folder and download sample CSVs
%sh
rm -r /dbfs/spark_lab
mkdir /dbfs/spark_lab
wget -O /dbfs/spark_lab/2019.csv https://raw.githubusercontent.com/Ch3rry-Pi3-Azure/DataBricks-Data-Analytics/refs/heads/main/data/2019_edited.csv
wget -O /dbfs/spark_lab/2020.csv https://raw.githubusercontent.com/Ch3rry-Pi3-Azure/DataBricks-Data-Analytics/refs/heads/main/data/2020_edited.csv
wget -O /dbfs/spark_lab/2021.csv https://raw.githubusercontent.com/Ch3rry-Pi3-Azure/DataBricks-Data-Analytics/refs/heads/main/data/2021_edited.csv

### <span style="color:#1f77b4">**Loading the CSV files into a dataframe**</span>

Read the CSVs into a Spark DataFrame and preview the rows.

- `spark.read.load` reads files into a DataFrame.
- `display` shows a sample in Databricks.



In [ ]:
# Load all CSVs into a Spark DataFrame
df = spark.read.load('spark_lab/*.csv', format='csv')
display(df.limit(100))

### <span style="color:#1f77b4">**Defining Schema for the dataframe**</span>

Apply an explicit schema so dates, numbers, and strings parse consistently.

- `StructType` defines the full schema structure.
- `StructField` defines each column name, type, and nullability.



In [ ]:
# Define an explicit schema for consistent types
from pyspark.sql.types import *
from pyspark.sql.functions import *
orderSchema = StructType([
    StructField("SalesOrderNumber", StringType()),
    StructField("SalesOrderLineNumber", IntegerType()),
    StructField("OrderDate", DateType()),
    StructField("CustomerName", StringType()),
    StructField("Email", StringType()),
    StructField("Item", StringType()),
    StructField("Quantity", IntegerType()),
    StructField("UnitPrice", FloatType()),
    StructField("Tax", FloatType())
])
df = spark.read.load('/spark_lab/*.csv', format='csv', schema=orderSchema)
display(df.limit(100))

### <span style="color:#1f77b4">**Cleaning the Data**</span>

Remove duplicates and recompute tax for consistency.

- `dropDuplicates` removes repeated rows.
- `withColumn` and `col` derive and cast columns.



In [ ]:
from pyspark.sql.functions import col
df = df.dropDuplicates()
df = df.withColumn('Tax', col('UnitPrice') * 0.08)
df = df.withColumn('Tax', col('Tax').cast("float"))

### <span style="color:#1f77b4">**Creating a new Dataframe**</span>

Build a customer-focused DataFrame with parsed names.

- `select` picks relevant columns.
- `split(...).getItem(...)` extracts first and last names.



In [ ]:
customers_df = df.select("CustomerName", "Email", "Item", "Quantity")
customers_df = customers_df.withColumn("FirstName", split(customers_df["CustomerName"], " ").getItem(0))
customers_df = customers_df.withColumn("LastName", split(customers_df["CustomerName"], " ").getItem(1))

display(customers_df)

### <span style="color:#1f77b4">**Counting distinct customer entries**</span>

Compare total rows to distinct rows to assess duplicates.

- `count` returns row counts.
- `distinct` removes duplicates before counting.



In [ ]:
print(customers_df.count())
print(customers_df.distinct().count())

### <span style="color:#1f77b4">**Creating a Product Sales Dataframe**</span>

Aggregate quantity sold by product.

- `groupBy` defines the grouping key.
- `sum` computes totals per product.



In [ ]:
productSales = df.select("Item", "Quantity").groupBy("Item").sum()
display(productSales)

### <span style="color:#1f77b4">**Aggregating Yealy Sales**</span>

Summarize sales counts by year.

- `year` extracts the year from dates.
- `groupBy` and `orderBy` build a yearly summary.



In [ ]:
yearlySales = df.select(year("OrderDate").alias("Year")).groupBy("Year").count().orderBy("Year")
display(yearlySales)